In [1]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize
import wandb
import timm
import torch
import avalanche
import torchvision

/home/grbagwe/miniconda3/envs/fedconlearn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchvision.datasets import MNIST,CIFAR10
from avalanche.benchmarks.datasets.dataset_utils import default_dataset_location

mnist_train = MNIST(
            root = default_dataset_location("mnist"),
            train=True,
            download=True,
            transform=ToTensor(),
        )
mnist_test = MNIST(
            root = default_dataset_location("mnist"),
            train=False,
            download=True,
            transform=ToTensor(),
        )

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
train_transform = Compose([
    Resize(224),
    ToTensor(),
])

test_transform = Compose([
    Resize(224),
    ToTensor(),
])


benchmark = avalanche.benchmarks.SplitCIFAR10(n_experiences=5, return_task_id=False, 
                       train_transform=train_transform,eval_transform = test_transform
                      )

Files already downloaded and verified
Files already downloaded and verified


In [17]:
benchmark.

<bound method Sequence.count of <avalanche.benchmarks.scenarios.classification_scenario.ClassificationStream object at 0x7f12b56ade50>>

In [8]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
loggers.append(WandBLogger(project_name="avalanche", run_name="fed-ewc-SplitCIFAR10"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: grbagwe. Use `wandb login --relogin` to force relogin


In [10]:
model = timm.models.vit_tiny_patch16_224(pretrained=True, num_classes=benchmark.n_classes)

In [11]:
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy = EWC(
    model,
    optimizer=SGD(model.parameters(), lr=0.001, momentum=0.9),
    criterion=CrossEntropyLoss(),
    ewc_lambda=0.4,
    train_mb_size=50, train_epochs=5,
    eval_mb_size=50,device='cuda:1',
    evaluator=eval_plugin)

In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream[:]))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values    